In [4]:
import serial

serialPort = serial.Serial(port = "/dev/ttyUSB1", baudrate=115200,
                           bytesize=8, timeout=2, stopbits=serial.STOPBITS_ONE)

In [5]:
#Converting the value to bytes.
print('Converting the integer value to byte format.')
import numpy as np
Byte_X_train=[]
to_fpga = range(20)
for i in to_fpga:
    Byte_X_train.append(bytes([i]))
    
print(Byte_X_train)

Converting the integer value to byte format.
[b'\x00', b'\x01', b'\x02', b'\x03', b'\x04', b'\x05', b'\x06', b'\x07', b'\x08', b'\t', b'\n', b'\x0b', b'\x0c', b'\r', b'\x0e', b'\x0f', b'\x10', b'\x11', b'\x12', b'\x13']


In [6]:
import time
import pdb
import itertools
from IPython.display import clear_output
import matplotlib.pyplot as plt
%matplotlib inline
fig = plt.figure
while(1):
    completedata_from_fpga = []
    serialPort.reset_input_buffer()
    serialPort.reset_output_buffer()  
    print('Data to be send to FPGA: ',list(to_fpga))
    print('------------------Sending to FPGA------------------------')
    print('Byte stream created. Sending now: {}'.format(Byte_X_train))
    print('Sent bytes of length {}.'.format(len(Byte_X_train)))
    serialString = ""                           # Used to hold data coming over UART
    for val in Byte_X_train:
        sent_val = serialPort.write(val)
    #print('Number of bytes sent is: {}. '.format(sent_val))
    count_received_data = 0
    while(1):
        # Wait until there is data waiting in the serial buffer
        if(serialPort.in_waiting > 0):
            serialString = serialPort.readlines()
            print('-----------Received data from FPGA.-----------')
            for string in serialString:
                received_data=list(string)
                completedata_from_fpga.append(received_data)                
                count_received_data = count_received_data + len(received_data) 
            print('Length of received data {}'.format(count_received_data))
            #Sanity check that sent and received pixels values are same.
            from_fpga = list(itertools.chain.from_iterable(completedata_from_fpga))
            print('Received data: ', from_fpga)
            for i,j in zip(to_fpga, from_fpga):
                if i!=j:
                    print(i,j)
            break
    print('Do you want to continue: press Y for yes and N for no.')
    continue_command = input()
    if continue_command == 'Y' or continue_command =='y':
        clear_output(wait=True)
    else:
        print('Thank you for using decision tree inference in FPGA.')
        break

Data to be send to FPGA:  [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19]
------------------Sending to FPGA------------------------
Byte stream created. Sending now: [b'\x00', b'\x01', b'\x02', b'\x03', b'\x04', b'\x05', b'\x06', b'\x07', b'\x08', b'\t', b'\n', b'\x0b', b'\x0c', b'\r', b'\x0e', b'\x0f', b'\x10', b'\x11', b'\x12', b'\x13']
Sent bytes of length 20.
-----------Received data from FPGA.-----------
Length of received data 20
Received data:  [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19]
Do you want to continue: press Y for yes and N for no.
N
Thank you for using decision tree inference in FPGA.
